In [1]:
from os import listdir
import shutil
from os.path import isfile, join
import os
import numpy as np
from skimage.transform import rescale, resize
from skimage.io import imsave, imread
from skimage import img_as_ubyte
from skimage.filters import threshold_otsu
import time
from skimage.util import invert

def get_binary(img):
    thresh = threshold_otsu(img)
    binary = img <= thresh
    return binary

def find_boundaries(img):
    img = invert(img)
    r, c = img.shape

    indexes = [0, 0, r - 1, c - 1]

    # minr
    for ri in range(r):
        if np.sum(img[ri, :]) > 0:
            indexes[0] = ri
            break

    # minc
    for ci in range(c):
        if np.sum(img[:, ci]) > 0:
            indexes[1] = ci
            break

    # maxr
    for ri in range(r - 1, 0, -1):
        if np.sum(img[ri, :]) > 0:
            indexes[2] = ri
            break

    # maxc
    for ci in range(c - 1, 0, -1):
        if np.sum(img[:, ci]) > 0:
            indexes[3] = ci
            break

    return indexes[0], indexes[1], indexes[2], indexes[3]

def crop_image(img, lt, rt, up, bt):
    return img[up: bt, lt:rt]

def rescale_to_fit_training_set_normal(digit):
    minr, minc, maxr, maxc = find_boundaries(digit)
    digit_sub = digit[minr:maxr, minc:maxc]
    r, c = digit_sub.shape
    # recale the image to fit inside a 20x20 box
    resized = rescale(digit_sub, np.minimum(22 / r, 22 / c), multichannel=False)

    # convert to binary
    binary = get_binary(resized)

    # fit it inside a 28 x 28 box, centered
    r, c = resized.shape
    r_dist = int((28 - r) / 2)
    c_dist = int((28 - c) / 2)
    digit = np.zeros((28, 28), dtype=np.int8)
    digit[r_dist:r_dist + r, c_dist:c_dist + c] = binary
    return digit

mypath = '/home/muthu/research/ml/by_class'
for f in listdir(mypath):
    path1 = join(mypath, f)
    if not isfile(path1):
        for d in listdir(path1):
            path2 = join(path1, d)
            if 'train_' in path2:
                for c in listdir(path2):
                    path3 = join(path2, c)
                    if isfile(path3) and d+'_' in path3:
                        newpath = path2.replace('/by_class/','/by_class_resized/')
                        if not isfile(join(newpath, c)):
                            image = rgb2gray(imread(path3))
                            resized = rescale_to_fit_training_set_normal(image)
                            newpath = path2.replace('/by_class/','/by_class_resized/')
                            if not os.path.exists(newpath):
                                os.makedirs(newpath)
                            imsave(join(newpath, c),resized)
                            print(join(newpath, c))

2019-11-27 21:44:16,704	WARNING worker.py:1268 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-11-27 21:44:16,709	INFO resource_spec.py:205 -- Starting Ray with 1.07 GiB memory available for workers and up to 0.54 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '192.168.31.76',
 'redis_address': '192.168.31.76:39234',
 'object_store_address': '/tmp/ray/session_2019-11-27_21-44-16_705932_73621/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-11-27_21-44-16_705932_73621/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-11-27_21-44-16_705932_73621'}

<Figure size 640x480 with 1 Axes>

In [6]:
from skimage.util import invert

def get_binary(img):
    thresh = threshold_otsu(img)
    binary = img <= thresh
    return binary

def find_boundaries(img):
    img = invert(img)
    r, c = img.shape

    indexes = [0, 0, r - 1, c - 1]

    # minr
    for ri in range(r):
        if np.sum(img[ri, :]) > 0:
            indexes[0] = ri
            break

    # minc
    for ci in range(c):
        if np.sum(img[:, ci]) > 0:
            indexes[1] = ci
            break

    # maxr
    for ri in range(r - 1, 0, -1):
        if np.sum(img[ri, :]) > 0:
            indexes[2] = ri
            break

    # maxc
    for ci in range(c - 1, 0, -1):
        if np.sum(img[:, ci]) > 0:
            indexes[3] = ci
            break

    return indexes[0], indexes[1], indexes[2], indexes[3]

def crop_image(img, lt, rt, up, bt):
    return img[up: bt, lt:rt]

@ray.remote
def rescale_to_fit_training_set(digit):
    minr, minc, maxr, maxc = find_boundaries(digit)
    digit_sub = digit[minr:maxr, minc:maxc]
    r, c = digit_sub.shape
    # recale the image to fit inside a 20x20 box
    resized = rescale(digit_sub, np.minimum(22 / r, 22 / c), multichannel=False)

    # convert to binary
    binary = get_binary(resized)

    # fit it inside a 28 x 28 box, centered
    r, c = resized.shape
    r_dist = int((28 - r) / 2)
    c_dist = int((28 - c) / 2)
    digit = np.zeros((28, 28), dtype=np.int8)
    digit[r_dist:r_dist + r, c_dist:c_dist + c] = binary
    return digit

def rescale_to_fit_training_set_normal(digit):
    minr, minc, maxr, maxc = find_boundaries(digit)
    digit_sub = digit[minr:maxr, minc:maxc]
    r, c = digit_sub.shape
    # recale the image to fit inside a 20x20 box
    resized = rescale(digit_sub, np.minimum(22 / r, 22 / c), multichannel=False)

    # convert to binary
    binary = get_binary(resized)

    # fit it inside a 28 x 28 box, centered
    r, c = resized.shape
    r_dist = int((28 - r) / 2)
    c_dist = int((28 - c) / 2)
    digit = np.zeros((28, 28), dtype=np.int8)
    digit[r_dist:r_dist + r, c_dist:c_dist + c] = binary
    return digit

In [14]:
from skimage.color import rgb2gray
import matplotlib.pyplot as plt 
import time

sample = '/Users/muthukrishnan/Downloads/by_class/4d/train_4d/train_4d_00008.png'

@ray.remote
def compress_and_save(image_path):
    img = imread(sample)
    gray = rgb2gray(img)
    subimg = rescale_to_fit_training_set_normal(gray)
    #save the image
    imsave('/tmp/'+str(time.time())+'.png', subimg)

def compress_and_save_normal(image_path):
    img = imread(sample)
    gray = rgb2gray(img)
    subimg = rescale_to_fit_training_set_normal(gray)
    #save the image
    imsave('/tmp/'+str(time.time())+'.png', subimg)
    
# img = imread(sample)
# gray = rgb2gray(img)
# object_id = rescale_to_fit_training_set.remote(gray)
# subimg = ray.get(object_id)
# plt.imshow(subimg)
# plt.show()
t1 = time.time()
for i in range(10, 50):
    compress_and_save_normal('/Users/muthukrishnan/Downloads/by_class/4d/train_4d/train_4d_000'+str(i)+'.png')

t2 = time.time()
print("time 1: "+str(t2-t1) )

process_ids = []
for i in range(10, 50):
    process_ids.append(compress_and_save.remote('/Users/muthukrishnan/Downloads/by_class/4d/train_4d/train_4d_000'+str(i)+'.png'))

ray.get(process_ids)
print("time 2: "+str(time.time()-t1) )



/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916527.6551082.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916527.668013.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916527.681775.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-pack

28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47


/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916527.861791.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916527.8777401.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916527.8970208.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-pac

28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47


/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.0693262.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.085655.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.110375.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-pack

28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47
28 47


/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.266696.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.276653.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.2865028.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/Users/muthukrishnan/Library/Python/3.7/lib/python/site-pack

28 47
28 47
28 47
time 1: 0.6591918468475342
(pid=73660) 28 47
(pid=73660) 28 47
(pid=73660) 28 47
(pid=73659) 28 47
(pid=73659) 28 47
(pid=73659) /Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.32189.png is a low contrast image
(pid=73659)   warn('%s is a low contrast image' % fname)
(pid=73659) Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
(pid=73659) /Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.361366.png is a low contrast image
(pid=73659)   warn('%s is a low contrast image' % fname)
(pid=73659) Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
(pid=73660) /Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.322597.png is a low contrast image
(pid

time 2: 0.8430438041687012
(pid=73660) 28 47
(pid=73660) 28 47
(pid=73660) 28 47
(pid=73659) 28 47
(pid=73659) 28 47
(pid=73659) 28 47
(pid=73659) 28 47
(pid=73659) /Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.443352.png is a low contrast image
(pid=73659)   warn('%s is a low contrast image' % fname)
(pid=73659) Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
(pid=73659) /Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.454587.png is a low contrast image
(pid=73659)   warn('%s is a low contrast image' % fname)
(pid=73659) Lossy conversion from int8 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
(pid=73659) /Users/muthukrishnan/Library/Python/3.7/lib/python/site-packages/skimage/io/_io.py:141: UserWarning: /tmp/1574916528.468978.png is a low 

In [43]:
import numpy as np
import pandas as pd
from skimage.io import imread

def create_columns(size):
    columns = ["label"]
    for i in range(size*size):
        columns.append("pixel"+str(i))
    
def imgtoarray(label, path):
    img = imread(path)
    w, h = img.shape
    flat = img.reshape(w*h)
    return np.append([label], flat)
    
arr = []
arr.append(imgtoarray("a",'/Users/muthukrishnan/programming/ml_research/resources/o1.png'))
arr.append(imgtoarray("a",'/Users/muthukrishnan/programming/ml_research/resources/o1.png'))
arr.append(imgtoarray("a",'/Users/muthukrishnan/programming/ml_research/resources/o1.png'))
arr.append(imgtoarray("a",'/Users/muthukrishnan/programming/ml_research/resources/o1.png'))

# Creating pandas dataframe from numpy array
dataset = pd.DataFrame(data=arr, columns=create_columns(128))